In [37]:
import warnings 
import pandas as pd
import numpy as np
import sys
sys.path.append("..")
import seaborn as sns
import matplotlib.pyplot as plt 
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import euclidean_distances

warnings.filterwarnings("ignore")

In [2]:
from scripts.data_read import read_data
from scripts.normalization import normalize_data

In [8]:
df = read_data("../data/experience_data.csv")
df.head()

,MSISDN/Number,Handset Type,Average TCP retransmission,Average RTT,Average throughput
0,33664962239,Samsung Galaxy A5 Sm-A520F,21569.572935,47.000000,67.0
1,33681854413,Samsung Galaxy J5 (Sm-J530),21569.572935,70.000000,42.0
2,33760627129,Samsung Galaxy A8 (2018),21569.572935,127.458589,15.0
3,33750343200,undefined,21569.572935,127.458589,88.0
4,33699795932,Samsung Sm-G390F,21569.572935,127.458589,15.0


In [11]:
df.isna().sum()

MSISDN/Number                 0
Handset Type                  0
Average TCP retransmission    0
Average RTT                   0
Average throughput            0
dtype: int64

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150001 entries, 0 to 150000
Data columns (total 5 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   MSISDN/Number               150001 non-null  int64  
 1   Handset Type                150001 non-null  object 
 2   Average TCP retransmission  150001 non-null  float64
 3   Average RTT                 150001 non-null  float64
 4   Average throughput          150001 non-null  float64
dtypes: float64(3), int64(1), object(1)
memory usage: 5.7+ MB


In [13]:
df_used = df.iloc[:,2:]

###### normalizing data 

In [18]:
standarlized_data = normalize_data.standardizer(df_used)

###### clustering 

In [19]:
kmeans = KMeans(n_clusters=3, random_state=0).fit(standarlized_data)

###### add clusters to dataframe 

In [20]:
df_used['clusters'] = kmeans.labels_

In [21]:
df_used

,Average TCP retransmission,Average RTT,Average throughput,clusters
0,21569.572935,47.000000,67.000000,2
1,21569.572935,70.000000,42.000000,2
2,21569.572935,127.458589,15.000000,0
3,21569.572935,127.458589,88.000000,0
4,21569.572935,127.458589,15.000000,0
...,...,...,...,...
149996,21569.572935,32.000000,117.000000,2
149997,21569.572935,29.000000,77.000000,2
149998,21569.572935,49.000000,90.000000,2
149999,21569.572935,42.000000,71.000000,2


###### aggregate per cluster to see less engaged cluster 

In [22]:
df_agg_clusters = df_used.groupby('clusters').agg({'Average TCP retransmission':'mean','Average RTT':'mean','Average throughput':'mean'}) 

In [23]:
df_agg_clusters

,Average TCP retransmission,Average RTT,Average throughput
clusters,,,
0,24744.482281,131.417720,2522.716211
1,25846.765661,80.810383,39972.360512
2,18527.426890,45.981731,2290.060516


cluster 1 is less engaged since it has high TCP and TP but less RTT 

In [27]:
df_norm = pd.DataFrame(standarlized_data,columns=df_used.iloc[:,:-1].columns.tolist())

In [29]:
df_norm['clusters'] =kmeans.labels_

In [30]:
df_norm

,Average TCP retransmission,Average RTT,Average throughput,clusters
0,0.004965,0.175373,0.000925,2
1,0.004965,0.261194,0.000580,2
2,0.004965,0.475592,0.000207,0
3,0.004965,0.475592,0.001214,0
4,0.004965,0.475592,0.000207,0
...,...,...,...,...
149996,0.004965,0.119403,0.001615,2
149997,0.004965,0.108209,0.001063,2
149998,0.004965,0.182836,0.001242,2
149999,0.004965,0.156716,0.000980,2


In [73]:
less_engaged_cluster_centroids = kmeans.cluster_centers_[1].tolist()

###### ecludian distance

In [64]:
X = df_norm.iloc[:1,:-1].to_numpy().flatten().reshape(-1,1)

array([[0.00496522],
       [0.17537313],
       [0.00092466]])

In [77]:
df_norm['Experience score'] = np.array(df_norm[['Average TCP retransmission','Average RTT','Average throughput']].sub(less_engaged_cluster_centroids, axis=1).pow(2).sum(axis=1).pow(.5)) * 100

In [78]:
df_norm

,Average TCP retransmission,Average RTT,Average throughput,clusters,Experience score
0,0.004965,0.175373,0.000925,2,56.498878
1,0.004965,0.261194,0.000580,2,55.255186
2,0.004965,0.475592,0.000207,0,57.828972
3,0.004965,0.475592,0.001214,0,57.732908
4,0.004965,0.475592,0.000207,0,57.828972
...,...,...,...,...,...
149996,0.004965,0.119403,0.001615,2,57.939722
149997,0.004965,0.108209,0.001063,2,58.353256
149998,0.004965,0.182836,0.001242,2,56.305985
149999,0.004965,0.156716,0.000980,2,56.938989
